
# **DATATHON UNIVERSITYHACK 2020**

## *Minsait Land Classification*

https://www.cajamardatalab.com/datathon-cajamar-universityhack-2020/retos/predictivo/

---

## Equipo ***Anacongas*** (*ESI, UCLM*)

> ![Logo](https://drive.google.com/uc?id=1j0Z02FzZDIuAJbZKvDe-84OE1az7O47r)


*   ### Antonio Manjavacas Lucas
*   ### Rubén Márquez Villalta
*   ### Diego Pedregal Hidalgo

#1. **CARGA DE DATOS**

In [1]:
!pip install imbalanced-learn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

from google.colab import drive

from sklearn import metrics
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

from imblearn.under_sampling import NearMiss
from imblearn.over_sampling import RandomOverSampler

from collections import Counter

from joblib import dump

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
TRAIN_DATA_PATH = os.path.join('.', 'drive', 'My Drive', 'Universityhack', 'Data', 'Modelar_UH2020.txt')

drive.mount('/content/drive')

df = pd.read_csv(TRAIN_DATA_PATH, index_col=0, sep='|')
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,X,Y,Q_R_4_0_0,Q_R_4_0_1,Q_R_4_0_2,Q_R_4_0_3,Q_R_4_0_4,Q_R_4_0_5,Q_R_4_0_6,Q_R_4_0_7,Q_R_4_0_8,Q_R_4_0_9,Q_R_4_1_0,Q_G_3_0_0,Q_G_3_0_1,Q_G_3_0_2,Q_G_3_0_3,Q_G_3_0_4,Q_G_3_0_5,Q_G_3_0_6,Q_G_3_0_7,Q_G_3_0_8,Q_G_3_0_9,Q_G_3_1_0,Q_B_2_0_0,Q_B_2_0_1,Q_B_2_0_2,Q_B_2_0_3,Q_B_2_0_4,Q_B_2_0_5,Q_B_2_0_6,Q_B_2_0_7,Q_B_2_0_8,Q_B_2_0_9,Q_B_2_1_0,Q_NIR_8_0_0,Q_NIR_8_0_1,Q_NIR_8_0_2,Q_NIR_8_0_3,Q_NIR_8_0_4,Q_NIR_8_0_5,Q_NIR_8_0_6,Q_NIR_8_0_7,Q_NIR_8_0_8,Q_NIR_8_0_9,Q_NIR_8_1_0,AREA,GEOM_R1,GEOM_R2,GEOM_R3,GEOM_R4,CONTRUCTIONYEAR,MAXBUILDINGFLOOR,CADASTRALQUALITYID,CLASE
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
35984B9C3E7CD9A1,2207357872,165920300,0.000000e+00,443.001300,616.001697,746.998401,872.996472,1009.000946,1159.002319,1342.002466,1572.003418,1943.686816,6512.787270,1.283357e-10,428.996347,574.000940,676.002960,778.003931,880.999176,997.000220,1130.001782,1311.997949,1626.996155,8627.098641,155.003632,402.000092,458.999939,510.000061,567.999939,632.000610,681.000916,726.109186,791.000269,902.999719,1980.172280,0.000013,892.016479,1244.014185,1497.998303,1731.204688,1944.003906,2160.996533,2410.995264,2719.999756,3180.000342,10951.926645,144.426900,0.557237,0.067249,0.057372,0.853127,2002,0.0,2,RESIDENTIAL
F9D04BF6D037F8FB,2189757160,165463267,5.879611e-05,443.899011,627.999060,770.001611,904.999988,1032.998474,1165.001636,1316.000110,1505.990771,1804.993469,6269.935600,1.352518e-11,433.998392,565.000110,662.996521,746.001318,828.998413,912.000330,1011.997162,1142.202637,1384.994580,5357.790999,120.000275,419.000000,472.999939,509.999808,541.999976,572.000000,602.999939,636.000122,673.000122,745.000000,3647.195402,0.000030,846.002728,1139.006982,1363.993860,1558.000513,1749.993713,1956.000464,2194.001978,2507.001416,3012.991675,7048.367637,38.342550,0.709884,0.125156,0.147929,1.181953,1949,1.0,8,RESIDENTIAL
B89D5711AFF8C423,2240147335,165690752,0.000000e+00,353.502274,523.003601,644.001831,760.997131,876.999634,1006.997498,1168.995544,1375.008179,1719.003235,5010.767611,0.000000e+00,379.001587,507.003815,600.000519,684.000488,768.997528,867.001587,992.991760,1165.998413,1443.000366,5240.232771,83.000389,363.000031,429.002502,473.000015,510.000092,549.000244,590.999939,635.999817,692.999817,796.498779,1947.739721,0.000005,807.001373,1124.002441,1365.009583,1570.995117,1768.992798,1978.989258,2228.999146,2561.999756,3088.000488,7013.073271,108.794384,0.517702,0.058268,0.081666,1.401552,1986,1.0,5,RESIDENTIAL
1C3478AC1522E7E4,2227146459,165934099,0.000000e+00,268.000613,376.999609,478.003784,575.001233,683.997742,809.005994,955.002960,1136.997144,1447.997290,5465.430281,6.341585e-06,310.999088,405.998712,485.000607,558.997925,641.001007,731.994189,838.004376,974.797485,1217.996167,5343.168796,0.001450,256.999487,295.999969,329.000092,366.000043,411.000549,457.000061,506.000000,565.999817,663.999817,1424.396092,0.002285,1200.994470,1476.001294,1678.005042,1848.001782,2022.988464,2211.993506,2433.003687,2740.996582,3349.986328,6216.880538,155.224455,0.450871,0.053591,0.054201,1.011382,1999,1.0,2,RESIDENTIAL
4D12AA5009064345,2212350459,165681791,0.000000e+00,318.997910,492.003845,632.999634,757.002197,882.999908,1019.008911,1175.993103,1401.000488,1756.994141,7376.529218,0.000000e+00,284.000534,421.002472,526.003815,618.998779,705.999268,801.005432,912.999451,1069.996094,1348.497314,7460.394534,193.000626,431.505035,462.999939,492.000061,519.000427,547.000061,578.999878,617.999603,662.999939,750.000488,1940.403140,0.000000,529.004425,790.996704,999.001617,1183.995239,1355.992920,1535.998779,1725.498962,1954.991455,2315.991455,7092.767616,1789.873366,0.458819,0.012858,0.019936,1.550478,1966,8.0,6,RESIDENTIAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C5FD596EB22DB167,2225843339,165482267,2.183254e-11,427.999246,604.004626,742.996454,866.005859,990

#2. **LIMPIEZA DE DATOS**

Eliminamos las filas con valores nulos:

In [3]:
df.isna().values.sum()

40

In [0]:
df.dropna(inplace=True)

Modificamos los identificadores por defecto:

In [0]:
df.reset_index(inplace=True)
df.drop(columns=['ID'], inplace=True)

Convertimos el campo `MAXBUILDINGFLOOR` en *Integer*:

In [0]:
df['MAXBUILDINGFLOOR'] = df['MAXBUILDINGFLOOR'].astype(np.int64)

Reasignamos los órdenes de calidad catastral (`CADASTRALQUALITYID`) considerando las categorías *A*, *B* y *C*:

* A &xrarr; 1 (mayor calidad)
* B &xrarr; 2
* C &xrarr; 3
* 1 &xrarr; 4
* 2 &xrarr; 5
* 3 &xrarr; 6
* ...
* 8 &xrarr; 11
* 9 &xrarr; 12 (menor calidad)

In [0]:
df['CADASTRALQUALITYID'].replace(['A','B','C',1,2,3,4,5,6,7,8,9], [1,2,3,4,5,6,7,8,9,10,11,12], inplace=True)
df['CADASTRALQUALITYID'] = df['CADASTRALQUALITYID'].astype(np.int64)

Codificamos el campo `CLASE` como valor numérico:

* AGRICULTURE &xrarr; 0
* INDUSTRIAL &xrarr; 1
* OFFICE &xrarr; 2
* OTHER &xrarr; 3
* PUBLIC &xrarr; 4
* RESIDENTIAL &xrarr; 5
* RETAIL &xrarr; 6

In [0]:
df['CLASE'].replace(['AGRICULTURE', 'INDUSTRIAL', 'OFFICE', 'OTHER', 'PUBLIC', 'RESIDENTIAL', 'RETAIL'], [0,1,2,3,4,5,6], inplace=True)
df['CLASE'] = pd.Categorical(df.CLASE)

Añadimos la media (`mean_red`, `mean_green`, `mean_blue`, `mean_nir`), valores máximos y mínimos (`max_red`, `max_green`, `max_blue`, `max_nir`, `min_red`, `min_green`, `min_blue`, `min_nir`) y desviación típica (`std_red`, `std_green`, `std_blue`, `std_nir`,) de los deciles para cada canal de color:

In [0]:
# Means
df['mean_red'] = df.loc[:, 'Q_R_4_0_0':'Q_R_4_1_0'].mean(axis=1)
df['mean_green'] = df.loc[:, 'Q_G_3_0_0':'Q_G_3_1_0'].mean(axis=1)
df['mean_blue'] = df.loc[:, 'Q_B_2_0_0':'Q_B_2_1_0'].mean(axis=1)
df['mean_nir'] = df.loc[:, 'Q_NIR_8_0_0':'Q_NIR_8_1_0'].mean(axis=1)

# Standard desviations
df['std_red'] = df.loc[:, 'Q_R_4_0_0':'Q_R_4_1_0'].std(axis=1)
df['std_green'] = df.loc[:, 'Q_G_3_0_0':'Q_G_3_1_0'].std(axis=1)
df['std_blue'] = df.loc[:, 'Q_B_2_0_0':'Q_B_2_1_0'].std(axis=1)
df['std_nir'] = df.loc[:, 'Q_NIR_8_0_0':'Q_NIR_8_1_0'].std(axis=1)

# Maximums
df['max_red'] = df.loc[:, 'Q_R_4_0_0':'Q_R_4_1_0'].max(axis=1)
df['max_green'] = df.loc[:, 'Q_G_3_0_0':'Q_G_3_1_0'].max(axis=1)
df['max_blue'] = df.loc[:, 'Q_B_2_0_0':'Q_B_2_1_0'].max(axis=1)
df['max_nir'] = df.loc[:, 'Q_NIR_8_0_0':'Q_NIR_8_1_0'].max(axis=1)

# Minimums
df['min_red'] = df.loc[:, 'Q_R_4_0_0':'Q_R_4_1_0'].min(axis=1)
df['min_green'] = df.loc[:, 'Q_G_3_0_0':'Q_G_3_1_0'].min(axis=1)
df['min_blue'] = df.loc[:, 'Q_B_2_0_0':'Q_B_2_1_0'].min(axis=1)
df['min_nir'] = df.loc[:, 'Q_NIR_8_0_0':'Q_NIR_8_1_0'].min(axis=1)

Incluimos el campo `RESIDENTIAL` para diferenciar instancias de clase residencial de las no residenciales:

In [0]:
df['RESIDENTIAL'] = (df['CLASE'] == 5).astype(int)
df['RESIDENTIAL'] = pd.Categorical(df.RESIDENTIAL)

Mostramos el *dataframe* final:

In [11]:
df

,X,Y,Q_R_4_0_0,Q_R_4_0_1,Q_R_4_0_2,Q_R_4_0_3,Q_R_4_0_4,Q_R_4_0_5,Q_R_4_0_6,Q_R_4_0_7,Q_R_4_0_8,Q_R_4_0_9,Q_R_4_1_0,Q_G_3_0_0,Q_G_3_0_1,Q_G_3_0_2,Q_G_3_0_3,Q_G_3_0_4,Q_G_3_0_5,Q_G_3_0_6,Q_G_3_0_7,Q_G_3_0_8,Q_G_3_0_9,Q_G_3_1_0,Q_B_2_0_0,Q_B_2_0_1,Q_B_2_0_2,Q_B_2_0_3,Q_B_2_0_4,Q_B_2_0_5,Q_B_2_0_6,Q_B_2_0_7,Q_B_2_0_8,Q_B_2_0_9,Q_B_2_1_0,Q_NIR_8_0_0,Q_NIR_8_0_1,Q_NIR_8_0_2,Q_NIR_8_0_3,Q_NIR_8_0_4,Q_NIR_8_0_5,Q_NIR_8_0_6,Q_NIR_8_0_7,Q_NIR_8_0_8,Q_NIR_8_0_9,Q_NIR_8_1_0,AREA,GEOM_R1,GEOM_R2,GEOM_R3,GEOM_R4,CONTRUCTIONYEAR,MAXBUILDINGFLOOR,CADASTRALQUALITYID,CLASE,mean_red,mean_green,mean_blue,mean_nir,std_red,std_green,std_blue,std_nir,max_red,max_green,max_blue,max_nir,min_red,min_green,min_blue,min_nir,RESIDENTIAL
0,2207357872,165920300,0.000000e+00,443.001300,616.001697,746.998401,872.996472,1009.000946,1159.002319,1342.002466,1572.003418,1943.686816,6512.787270,1.283357e-10,428.996347,574.000940,676.002960,778.003931,880.999176,997.000220,1130.001782,1311.997949,1626.996155,8627.098641,155.003632,402.000092,458.999939,510.000061,567.999939,632.000610,681.000916,726.109186,791.000269,902.999719,1980.172280,0.000013,892.016479,1244.014185,1497.998303,1731.204688,1944.003906,2160.996533,2410.995264,2719.999756,3180.000342,10951.926645,144.426900,0.557237,0.067249,0.057372,0.853127,2002,0,2,5,1474.316464,1548.281646,709.753331,2612.105101,1755.391392,2388.497594,468.015083,2902.546521,6512.787270,8627.098641,1980.172280,10951.926645,0.000000e+00,1.283357e-10,155.003632,0.000013,1
1,2189757160,165463267,5.879611e-05,443.899011,627.999060,770.001611,904.999988,1032.998474,1165.001636,1316.000110,1505.990771,1804.993469,6269.935600,1.352518e-11,433.998392,565.000110,662.996521,746.001318,828.998413,912.000330,1011.997162,1142.202637,1384.994580,5357.790999,120.000275,419.000000,472.999939,509.999808,541.999976,572.000000,602.999939,636.000122,673.000122,745.000000,3647.195402,0.000030,846.002728,1139.006982,1363.993860,1558.000513,1749.993713,1956.000464,2194.001978,2507.001416,3012.991675,7048.367637,38.342550,0.709884,0.125156,0.147929,1.181953,1949,1,8,5,1440.165435,1185.998224,812.745053,2125.032818,1678.773014,1431.888085,954.291968,1826.806363,6269.935600,5357.790999,3647.195402,7048.367637,5.879611e-05,1.352518e-11,120.000275,0.000030,1
2,2240147335,165690752,0.000000e+00,353.502274,523.003601,644.001831,760.997131,876.999634,1006.997498,1168.995544,1375.008179,1719.003235,5010.767611,0.000000e+00,379.001587,507.003815,600.000519,684.000488,768.997528,867.001587,992.991760,1165.998413,1443.000366,5240.232771,83.000389,363.000031,429.002502,473.000015,510.000092,549.000244,590.999939,635.999817,692.999817,796.498779,1947.739721,0.000005,807.001373,1124.002441,1365.009583,1570.995117,1768.992798,1978.989258,2228.999146,2561.999756,3088.000488,7013.073271,108.794384,0.517702,0.058268,0.081666,1.401552,1986,1,5,5,1221.752412,1149.838985,642.840122,2137.005749,1344.470812,1411.242332,471.633303,1825.051723,5010.767611,5240.232771,1947.739721,7013.073271,0.000000e+00,0.000000e+00,83.000389,0.000005,1
3,2227146459,165934099,0.000000e+00,268.000613,376.999609,478.003784,575.001233,683.997742,809.005994,955.002960,1136.997144,1447.997290,5465.430281,6.341585e-06,310.999088,405.998712,485.000607,558.997925,641.001007,731.994189,838.004376,974.797485,1217.996167,5343.168796,0.001450,256.999487,295.999969,329.000092,366.000043,411.000549,457.000061,506.000000,565.999817,663.999817,1424.396092,0.002285,1200.994470,1476.001294,1678.005042,1848.001782,2022.988464,2211.993506,2433.003687,2740.996582,3349.986328,6216.880538,155.224455,0.450871,0.053591,0.054201,1.011382,1999,1,2,5,1108.766968,1046.178033,479.672489,2288.986725,1501.493325,1462.943373,359.009949,1565.493384,5465.430281,5343.168796,1424.396092,6216.880538,0.000000e+00,6.341585e-06,0.001450,0.002285,1
4,2212350459,165681791,0.000000e+00,318.997910,492.003845,632.999634,757.002197,882.999908,1019.008911,1175.993103,1401.000488,1756.994141,7376.529218,0.000000e+00,284.000534,421.002472

In [12]:
%cd /content/drive/My Drive/Universityhack/Modelos

/content/drive/My Drive/Universityhack/Modelos


#3. **PRIMER MODELO: RESIDENCIAL vs NO RESIDENCIAL**

##3.1. **Selección de características**

In [0]:
selected_features = [
  'mean_red', 'mean_green', 'mean_blue', 'mean_nir',
  'AREA',
  'GEOM_R1', 'GEOM_R2', 'GEOM_R3', 'GEOM_R4',
  'CONTRUCTIONYEAR',
  'MAXBUILDINGFLOOR',
  'CADASTRALQUALITYID'
]

##3.2. **Obtención del conjunto de entrada y salida**

División del dataframe en entrada y salida del modelo:

In [14]:
X_train = df[selected_features]
y_train = df['RESIDENTIAL']

print('X_train: ', X_train.shape)
print('y_train: ', y_train.shape)

X_train:  (103210, 12)
y_train:  (103210,)


Aplicamos *oversampling* para balancear las clases:

In [15]:
ros = RandomOverSampler('minority', random_state=123)
X_train_res, y_train_res = ros.fit_sample(X_train, y_train)
 
print('Distribution before resampling {}'.format(Counter(y_train)))
print('Distribution after resampling {}'.format(Counter(y_train_res)))

Distribution before resampling Counter({1: 90173, 0: 13037})
Distribution after resampling Counter({1: 90173, 0: 90173})


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [16]:
print('X_train: ', X_train_res.shape)
print('y_train: ', y_train_res.shape)

X_train:  (180346, 12)
y_train:  (180346,)


##3.3. **Construcción del modelo y entrenamiento**

Seleccionamos los parámetros de nuestro modelo:

In [0]:
MIN_SAMPLES_SPLIT = 120
MIN_SAMPLES_LEAF = 10
MAX_DEPTH = 412

CRITERION = 'gini'

Construimos y entrenamos el modelo:

In [18]:
clf = tree.DecisionTreeClassifier(criterion=CRITERION, 
                                  min_samples_split=MIN_SAMPLES_SPLIT, 
                                  min_samples_leaf=MIN_SAMPLES_LEAF, 
                                  max_depth=MAX_DEPTH)

clf.fit(X_train_res, y_train_res)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=412, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=120,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

Guardamos el modelo:

In [19]:
dump(clf, 'model1.joblib')

['model1.joblib']

#4. **SEGUNDO MODELO: CLASIFICACIÓN DE NO RESIDENCIALES**

## 4.1. **Preparación del nuevo dataset**

El segundo *dataframe* contendrá los edificios clasificados como NO RESIDENCIALES:

In [20]:
df2 = df

df2['RESIDENTIAL_PREDICTED'] = clf.predict(df[selected_features])

df2.drop(df2[df2.RESIDENTIAL_PREDICTED == 1].index, inplace=True)

df2

,X,Y,Q_R_4_0_0,Q_R_4_0_1,Q_R_4_0_2,Q_R_4_0_3,Q_R_4_0_4,Q_R_4_0_5,Q_R_4_0_6,Q_R_4_0_7,Q_R_4_0_8,Q_R_4_0_9,Q_R_4_1_0,Q_G_3_0_0,Q_G_3_0_1,Q_G_3_0_2,Q_G_3_0_3,Q_G_3_0_4,Q_G_3_0_5,Q_G_3_0_6,Q_G_3_0_7,Q_G_3_0_8,Q_G_3_0_9,Q_G_3_1_0,Q_B_2_0_0,Q_B_2_0_1,Q_B_2_0_2,Q_B_2_0_3,Q_B_2_0_4,Q_B_2_0_5,Q_B_2_0_6,Q_B_2_0_7,Q_B_2_0_8,Q_B_2_0_9,Q_B_2_1_0,Q_NIR_8_0_0,Q_NIR_8_0_1,Q_NIR_8_0_2,Q_NIR_8_0_3,Q_NIR_8_0_4,Q_NIR_8_0_5,Q_NIR_8_0_6,Q_NIR_8_0_7,Q_NIR_8_0_8,Q_NIR_8_0_9,Q_NIR_8_1_0,AREA,GEOM_R1,GEOM_R2,GEOM_R3,GEOM_R4,CONTRUCTIONYEAR,MAXBUILDINGFLOOR,CADASTRALQUALITYID,CLASE,mean_red,mean_green,mean_blue,mean_nir,std_red,std_green,std_blue,std_nir,max_red,max_green,max_blue,max_nir,min_red,min_green,min_blue,min_nir,RESIDENTIAL,RESIDENTIAL_PREDICTED
4,2212350459,165681791,0.000000e+00,318.997910,492.003845,632.999634,757.002197,882.999908,1019.008911,1175.993103,1401.000488,1756.994141,7376.529218,0.000000e+00,284.000534,421.002472,526.003815,618.998779,705.999268,801.005432,912.999451,1069.996094,1348.497314,7460.394534,193.000626,431.505035,462.999939,492.000061,519.000427,547.000061,578.999878,617.999603,662.999939,750.000488,1940.403140,0.000000e+00,529.004425,790.996704,999.001617,1183.995239,1355.992920,1535.998779,1725.498962,1954.991455,2315.991455,7092.767616,1789.873366,0.458819,0.012858,0.019936,1.550478,1966,8,6,5,1437.593578,1286.263427,654.173563,1771.294470,2030.964843,2081.176715,450.052681,1883.124040,7376.529218,7460.394534,1940.403140,7092.767616,0.000000e+00,0.000000e+00,193.000626,0.000000e+00,1,0
6,2141448290,165496766,0.000000e+00,274.999353,416.001074,561.002997,706.001086,862.001343,1040.000391,1266.006592,1581.597119,2051.001392,6848.847191,1.367917e-05,327.000052,445.999768,556.999951,670.003247,793.000763,931.999792,1117.894409,1390.999951,1829.993933,6828.828872,28.000446,167.999600,250.000433,312.999789,378.999908,465.999847,612.999939,792.998962,1031.999658,1329.998962,3084.992882,0.000000e+00,1257.005884,1542.010864,1727.000659,1889.002905,2050.001587,2214.999756,2399.986523,2636.997656,3041.991968,7285.159019,373.432985,0.251622,0.029647,0.022728,0.766600,1982,1,4,5,1418.859867,1353.883705,768.817312,2367.650620,1896.291927,1885.889985,860.471441,1816.450934,6848.847191,6828.828872,3084.992882,7285.159019,0.000000e+00,1.367917e-05,28.000446,0.000000e+00,1,0
7,2212938608,165575238,1.308316e-10,405.999329,561.998108,694.002716,832.001648,969.999817,1115.003906,1277.000732,1479.002441,1786.991699,8377.113855,5.399681e-11,353.501877,476.000641,573.002747,667.999390,762.001892,862.998169,975.997986,1120.999146,1348.002319,8023.187183,213.000015,450.999359,518.000183,552.999939,576.000122,593.999939,611.999817,635.000061,665.999817,715.999817,2284.645262,8.301559e-11,543.997894,793.000061,996.000458,1174.005249,1340.004150,1520.997314,1708.000305,1945.998535,2276.990967,8126.684713,489.776545,0.349491,0.025246,0.028265,1.119589,1952,7,4,5,1590.828568,1378.517395,710.785848,1856.879968,2306.255434,2234.859054,538.861503,2177.337003,8377.113855,8023.187183,2284.645262,8126.684713,1.308316e-10,5.399681e-11,213.000015,8.301559e-11,1,0
9,2267210836,165899063,1.098887e-03,454.999338,573.005005,668.002612,752.999573,853.000122,1005.999158,1201.701685,1448.999097,1835.893945,5082.265959,9.081191e-06,433.000275,561.000623,640.306604,712.000574,791.998413,903.000171,1060.999878,1270.998755,1634.014819,5159.563755,28.000694,268.000031,323.000159,375.999643,433.000000,501.000000,591.002417,715.999780,858.999988,1015.000116,2043.290997,7.619475e-03,1239.002832,1485.999219,1697.999463,1891.001611,2079.999878,2289.000439,2548.999268,2960.005029,3724.899341,5997.459077,333.617230,0.860099,0.046252,0.055740,1.205122,1990,1,6,1,1261.533417,1196.989443,650.299439,2355.852162,1361.092410,1383.237480,539.453778,1543.415612,5082.265959,5159.563755,2043.290997,5997.459077,1.098887e-03,9.081191e-06,28.000694,7.619475e-03,0,0
12,2230759603,165513206,0.000000e+00,344.003510,537.000488,678.994141,800.003784,926.997253,1070.998901,1241.997375,1467

##4.2. **Selección de características**

In [0]:
selected_features_2 = [
  'X','Y',
  'max_red', 'max_green', 'max_blue', 'max_nir',
  'AREA',
  'GEOM_R1', 'GEOM_R2', 'GEOM_R3', 'GEOM_R4',
  'CONTRUCTIONYEAR',
  'MAXBUILDINGFLOOR',
  'CADASTRALQUALITYID'
]

##4.3. **Obtención del conjunto de entrada y salida**

División del dataframe en entrada y salida del modelo:

In [22]:
X_train = df2[selected_features_2]
y_train = df2['CLASE']

print('X_train: ', X_train.shape)
print('y_train: ', y_train.shape)

X_train:  (23470, 14)
y_train:  (23470,)


##4.4. **Construcción del modelo y entrenamiento**

Seleccionamos los parámetros de nuestro modelo:

In [0]:
N_ESTIMATORS = 128
CRITERION = 'entropy'
MAX_DEPTH = None
MAX_FEATURES = 'sqrt'
MIN_SAMPLES_LEAF = 1
MIN_SAMPLES_SPLIT = 5
BOOTSTRAP = True
CLASS_WEIGHT = 'balanced'

Construimos y entrenamos el segundo modelo:

In [24]:
clf = RandomForestClassifier(n_estimators=N_ESTIMATORS, criterion=CRITERION, 
                                max_depth=MAX_DEPTH, max_features=MAX_FEATURES, 
                                min_samples_leaf=MIN_SAMPLES_LEAF, 
                                min_samples_split=MIN_SAMPLES_SPLIT,
                                bootstrap=BOOTSTRAP, n_jobs=-1, 
                                class_weight=CLASS_WEIGHT)

clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='entropy', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=128,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

Guardamos el segundo modelo:

In [25]:
dump(clf, 'model2.joblib')

['model2.joblib']